In [ ]:
from pyomo.environ import *
import pandas as pd


In [ ]:
# Data
schools = ['S1', 'S2', 'S3', 'S4', 'S5']
grades = ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11', 'G12']


In [ ]:
distance_data = {
    'S1': [0, 1, 2.5, 1.1, 0.5],
    'S2': [1, 0, 0.8, 1.7, 3],
    'S3': [2.5, 0.8, 0, 3, 5],
    'S4': [1.1, 1.7, 3, 0, 1.2],
    'S5': [0.5, 3, 5, 1.2, 0]}

distance_df = pd.DataFrame(distance_data, index=schools)


In [ ]:
enrolment_data = {
    'S1': [10, 12, 8, 9, 4, 0, 0, 0, 0, 0, 0, 0],
    'S2': [15, 13, 17, 12, 9, 0, 0, 0, 0, 0, 0, 0],
    'S3': [3, 22, 9, 12, 19, 8, 4, 6, 11, 8, 10, 14],
    'S4': [8, 12, 15, 8, 1, 9, 3, 11, 5, 7, 16, 9],
    'S5': [0, 0, 15, 8, 1, 9, 3, 11, 0, 0, 0, 0]}

enrolment_df = pd.DataFrame(enrolment_data, index=grades)


In [ ]:
M = 1
T = 50

In [ ]:
# Type of Model chosen.
model = ConcreteModel()

Here, decision variables are as follows:

- $d_{i,j}$ be the distance between school $i$ and school $j$.
- $x_i$ be a binary decision variable indicating whether school $i$ is selected.
- $y_{i,j,k}$ be a binary decision variable indicating whether the pair of schools $i$ and $j$ is selected for grade $k$.




In [ ]:
# Decision Variables
model.x = Var(schools, domain=Binary)
model.y = Var(schools, schools, grades, domain=Binary)

The objective function is to minimize the total distance, which can be expressed as:

$\text{minimize} \quad \sum_{i} \sum_{j} \sum_{k} d_{i,j} \cdot y_{i,j,k}$


In [ ]:
model.obj = Objective(expr=sum(distance_df.loc[i, j] * model.y[i, j, k] 
                                for i in schools for j in schools for k in grades), sense=minimize)


In [ ]:
model.constraints = ConstraintList()

Each grade should be relocated to only one school.

$\sum_{j } y_{i,j,k} = 1$


In [ ]:
for i in schools:
    for k in grades:
        model.constraints.add(sum(model.y[i, j, k] for j in schools) == 1)


Total enrolment of the relocated grades should meet the minimum threshold if the school is closed.

$\sum_{j=1}^{n} \sum_{k=1}^{12} E_{ij} \cdot y_{ijk} \geq T \cdot x_{ij} \quad \forall i$



In [ ]:
for i in schools:
    model.constraints.add(sum(enrolment_df.loc[k, i] * model.y[i, j, k] for j in schools for k in grades) 
                          >= T * model.x[i])


A school can be closed only if all grades it serves can be relocated within the maximum distance.

$\sum_{k=1}^{12} y_{ijk} \leq M \cdot x_{ij} \quad \forall i$



In [ ]:
M = len(schools)
for i in schools:
    model.constraints.add(sum(model.y[i, j, k] for j in schools for k in grades) <= M * model.x[i])


In [ ]:
# Solve
solver = SolverFactory('glpk')
results = solver.solve(model)

# Display Results after solving the model
if results.solver.termination_condition == TerminationCondition.optimal:
    print("Total travel distance:", model.obj())
    print("Decision on School Closure:")
    for i in schools:
        if model.x[i].value == 1:
            print("Shut the school.", i)
        else:
            print("Continue the school", i)

    print("Decision on relocation:")
    for i in schools:
        for j in schools:
            for k in grades:
                if model.y[i, j, k].value == 1:
                    print("Relocate grade", k, "from school", i, "to school", j)
else:
    print("No solutions found/ No schools can be closed.")
